#Langchain

In this lab, we will introduce [Langchain](https://python.langchain.com/docs/get_started/introduction), a framework for developing applications powered by language models.

Langchain supports Python and Javascript / Typescript. For this lab, we will use Python.

We'll start by importing the `AzureOpenAI` specific components from the `langchain` package, including models and schemas for interacting with the API.

In [1]:
from langchain_openai import AzureChatOpenAI

As with all the other labs, we'll need to provide our API key and endpoint details, so we'll load them from our `.env` file.

In [2]:
import os
import openai
from dotenv import load_dotenv

if load_dotenv():
    print("Found Azure OpenAI Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("No file .env found")

Found Azure OpenAI Endpoint: https://ai-demos-aoai-kedamm.openai.azure.com/


Next, we'll configure Langchain by providing the Azure OpenAI deployment name. Langchain will automatically retrieve details for the Azure OpenAI endpoint and version from the environment variables we've set above.

In [3]:
# Create an instance of Azure OpenAI
llm = AzureChatOpenAI(
    azure_deployment = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
)

## Send a prompt to Azure OpenAI using Langchain

We're now ready to send a request to Azure OpenAI. To do this, we invoke the `llm` instance we created above and pass in the prompt.

In [4]:
r = llm.invoke("What things could I make with a Raspberry Pi?")

# Print the response
print(r.content)

A Raspberry Pi is a versatile little computer that can be used for a wide range of projects, from simple to complex. Here are some ideas for things you could make with a Raspberry Pi:

1. **Media Center**: Use software like Kodi or Plex to turn your Raspberry Pi into a home theater PC, capable of streaming or playing locally stored media.

2. **Retro Gaming Console**: Install RetroPie or another emulator to play classic video games from consoles like the NES, SNES, Sega Genesis, and more.

3. **Home Automation System**: Create a smart home hub to control lights, thermostats, and other smart devices using platforms like Home Assistant or OpenHAB.

4. **Web Server**: Set up a web server using Apache or Nginx to host your personal website or blog.

5. **Network Attached Storage (NAS)**: Turn your Raspberry Pi into a file server to share files across your home network.

6. **Personal Cloud**: Use software like Nextcloud to create your own private cloud storage solution.

7. **VPN Server**:

Compared to using the OpenAI Python library as we did in the previous lab, Langchain further simplified the process of interacting with the LLM by reducing it to a `llm.invoke` call.

## Using templates and chains

We've seen that we can use Langchain to interact with the LLM and it's a little easier to work with than the OpenAI Python library. However, that's just the start of how Langchain makes it easier to work with LLM's. Most OpenAI models are designed to be interacted with using a Chat style interface, where you provide a persona or system prompt which helps the LLM understand the context of the conversation. This will then be sent to the LLM along with the user's request.

So that you don't have to setup the persona / system prompt every time you want to interact with the LLM, Langchain provides the concept of Templates. Templates are a way to define the persona and system prompt once and then reuse them across multiple interactions.

In [5]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a chatbot that helps people generate ideas for their next project. You can help them brainstorm ideas, come up with a plan, or even help them with their project."),
    ("user", "{input}")
])

Above we've defined a "system" message which will tell the LLM how we're expecting it to respond, and an `{input}` placeholder for the user's prompt.

Next, we define a chain. A chain allows us to define a sequence of operations that we want to perform. In this case, we're defining a simple chain that will take the prompt we've defined above and send it to the LLM.

In [6]:
chain = prompt | llm

Now, we can invoke the chain in a similar fashion to how to invoked the LLM earlier. This time, we're passing in the user's input as a parameter to the chain, which will replace the `{input}` placeholder in the prompt.

In [7]:
chain.invoke({"input": "I've just purchased a Raspberry Pi and I'm looking for a project to work on. Can you help me brainstorm some ideas?"})

AIMessage(content='Absolutely, Raspberry Pi is a versatile tool for a wide range of projects. Here are some ideas to get you started:\n\n1. **Home Automation System**:\n   - Control lights, thermostats, and security systems.\n   - Integrate with voice assistants like Alexa or Google Assistant.\n   - Use sensors to create a smart garden that monitors soil moisture and waters plants automatically.\n\n2. **Media Center**:\n   - Set up a Kodi or Plex media server to stream movies and music to your devices.\n   - Create a retro gaming console using RetroPie to play classic games.\n\n3. **Weather Station**:\n   - Use sensors to collect data on temperature, humidity, and air pressure.\n   - Display the data on a web dashboard or mobile app.\n\n4. **Web Server**:\n   - Host a personal website or blog.\n   - Learn basic web development by setting up a LAMP (Linux, Apache, MySQL, PHP) server.\n\n5. **Network-wide Ad Blocker**:\n   - Install Pi-hole to block advertisements on all devices connecte

The result will be an `AIMessage` object, which contains the response from the LLM.

Let's enhance the chain further to get it to parse the output from the LLM and extract the text from the response. First, we define an output parser.

In [8]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Next, we redefine our chain to include the output parser. So now when we invoke the chain, it will 

- Take the prompt template and add the user's input
- Send the prompt to the LLM
- Parse the response from the LLM and extract the text

In [9]:
chain = prompt | llm | output_parser

Now let's invoke the chain again with the same prompt as before.

In [10]:
chain.invoke({"input": "I've just purchased a Raspberry Pi and I'm looking for a project to work on. Can you help me brainstorm some ideas?"})

"Absolutely! Raspberry Pi is a versatile platform, and there are countless projects you can explore depending on your interests and skill level. Here are some ideas to get you started:\n\n1. **Home Automation System**: Create a smart home hub that can control lights, thermostats, and other devices. Integrate it with voice assistants like Alexa or Google Assistant for added functionality.\n\n2. **Retro Gaming Console**: Use RetroPie or Recalbox to turn your Raspberry Pi into a retro gaming console that can play old-school games from consoles like NES, SNES, and Sega Genesis.\n\n3. **Media Center**: Set up a Kodi or Plex media server to stream movies, TV shows, and music to your devices. This could include setting up a local network storage option.\n\n4. **Personal Web Server**: Host your own website or blog using software like WordPress or Ghost. This is a great way to learn about web hosting and server management.\n\n5. **Network Ad Blocker**: Install Pi-hole to create a network-wide a

This time, you should only get a string containing the text from the response.

We can do much more powerful things with chains than simply setting up and passing prompts to the LLM and parsing the results. We can augment the prompt with external data retrieved from a database, we could add conversation history to provide context for a chatbot, or we could even chain multiple LLMs together to create a more powerful model. We'll explore some of these ideas in future labs.

## Summary

Langchain is an example of an AI orchestrator. It provides an alternative method to the raw API or using an SDK package to access the AI models, but on top of that can provide additional integrations, deal with issues related to rate limiting of the API and provide an abstraction layer over potentially complex operations. We'll get into those more complex use cases in later labs.

## Up Next

In the next lab, we'll look at another AI Orchestrator - Semantic Kernel.

## Next Section

📣 [Semantic Kernel](../04-SemanticKernel/semantickernel.ipynb)